In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sys
import pickle 
from collections import Counter

# Initialize linear regression
model = LinearRegression()

# Set random seed 
random_state = 12

# Set minimum number of cells 
min_cells = 7

# Read Slide-seq transformed coorindates
print("Loading coordinates")
df = pd.read_csv("data/transformed_coords_1to2.txt",
                 sep = "\t", header=None, names=["OR", "X", "Y", "EXP"])


# Average coordinates shared between the two experiments
df = df.groupby("OR").agg({"X" : "mean", "Y" : "mean", "EXP" : "sum"})

# This is just a quick and dirty way to remove the OR that falls of the slide without knowing its ID
df = df[df["Y"] < 0]

ORs = sorted(list(df.index))
print("{} ORs with Slide-seq coordinates".format(len(ORs)))

# Convert coordinates to a dictionary keyed on OR
coords = df[["X", "Y"]].to_dict("index")

# Fit scaler so it stores the min and max of the x and y coordinates
scaler_y = MinMaxScaler().fit(df[["X", "Y"]].values)

# Read in PCs
df = pd.read_csv("data/All_OSN_PC.txt", sep = "\t", index_col=0)

n_col = len(df.columns)
df.rename(columns={"OR_identity": "OR"}, inplace=True)

print(df.head())

# Only retain PCs for ORs with slide-seq coordinates
df = df[df["OR"].isin(ORs)]

ORs_to_exclude = [x for x in ORs if len(df[df["OR"] == x]) < min_cells]
print(ORs_to_exclude)
df = df[~df["OR"].isin(ORs_to_exclude)]


print("{} ORs with Slide-seq coordinates but no gene expression".format(len(ORs_to_exclude)))
ORs = list(df.OR.unique())
print("{} ORs remaining".format(len(ORs)))

# Z-score normalize PCs
scaler_x = StandardScaler().fit(df.values[:,0:(n_col - 1)])

# Generating train and eval set 
X_train = df.values[:,0:(n_col - 1)]
X_train = scaler_x.transform(X_train)

train_ors = np.array(df.iloc[:,(n_col - 1)])

# Use KNN to oversample less represented cells to remove any potential bias
# created by ORs with large numbers of cells
counter = Counter(train_ors)
oversample = SMOTE(random_state=random_state)
X_train, train_ors = oversample.fit_resample(X_train, train_ors)
counter = Counter(train_ors)
max_cells = np.max(list(counter.values()))


# Generate coordinate vectors from coordinate dictionary and ORs
y_train = []
for i in range(train_ors.shape[0]):
    y_train.append([coords[train_ors[i]]["X"], coords[train_ors[i]]["Y"]])
        

y_train = np.array(y_train)

# Scale coordinates
y_train = scaler_y.transform(y_train)

# Fit linear regression
model.fit(X_train, y_train)

# Pickle model
model_file = "data/model.pkl"
pickle.dump(model, open(model_file, 'wb'))

np.savetxt("data/coefficients.txt", np.transpose(model.coef_))

f_x, p_x = f_regression(X_train, y_train[:,0])
f_y, p_y = f_regression(X_train, y_train[:,1])

with open("data/coefficient_p_values.txt", "w") as f:
    for i in range(p_x.shape[0]):
        print("x", i+1, "{:e}".format(p_x[i]), file = f, sep="\t")
        
    for i in range(p_y.shape[0]):
        print("y", i+1, "{:e}".format(p_y[i]), file = f, sep="\t")



# Run lr on bootstrap samples to get prediction intervals
STDS_X = []
STDS_Y = []
print(len(ORs))
for i in range(100):
    bootstrap_ORs = np.random.choice(ORs, len(ORs))
    
    pc_arrays = []
    coord_arrays = []
    for x in bootstrap_ORs:
        indices = [j for j in range(len(train_ors)) if train_ors[j] == x]
        pc_arrays.append(X_train[indices,:])
        coord_arrays.append(y_train[indices,:])
        
    
    model = LinearRegression()
    model.fit(np.vstack(pc_arrays), np.vstack(coord_arrays))
    
    preds = model.predict(np.vstack(pc_arrays))
    x_preds = []
    y_preds = []
    x_true = []
    y_true = []
    coord_arrays = scaler_y.inverse_transform(np.vstack(coord_arrays))
    
    for j, x in enumerate(bootstrap_ORs):
        x_preds.append(np.mean(preds[j*max_cells:(j*max_cells + max_cells),0]))
        y_preds.append(np.mean(preds[j*max_cells:(j*max_cells + max_cells),1]))
        x_true.append(np.mean(coord_arrays[j*max_cells:(j*max_cells + max_cells),0]))
        y_true.append(np.mean(coord_arrays[j*max_cells:(j*max_cells + max_cells),1]))
        
        
    x_preds = np.array(x_preds)
    y_preds = np.array(y_preds)
    
    
    mean_preds = np.transpose(np.vstack((x_preds, y_preds)))
    mean_preds = scaler_y.inverse_transform(mean_preds)
    x_true = np.array(x_true)
    y_true = np.array(y_true)
    
    
    stdev_x = np.sqrt(1/(len(ORs)-2) * np.sum((x_true - mean_preds[:,0])**2))
    stdev_y = np.sqrt(1/(len(ORs)-2) * np.sum((y_true - mean_preds[:,1])**2))
    STDS_X.append(stdev_x)
    STDS_Y.append(stdev_y)
    
print(np.mean(STDS_X), np.mean(STDS_Y))

# Make predictions on all other ORs
df = pd.read_csv("data/All_OSN_PC.txt", sep = "\t", index_col=0)
df.rename(columns={"OR_identity": "OR"}, inplace=True)
n_col = len(df.columns)

model_file = "data/model.pkl"
model = pickle.load(open(model_file, 'rb'))

all_ORs = list(df.OR.unique())
ORs_of_interest = [x for x in all_ORs if len(df[df["OR"] == x]) >= min_cells]
print(len(ORs_of_interest))

with open("data/map_654.txt", "w") as f:
    for OR in ORs_of_interest:
        X = df[df["OR"] == OR].values[:,0:(n_col - 1)]
        n = X.shape[0]
        X = scaler_x.transform(X)
        pred = scaler_y.inverse_transform(model.predict(X))
        pred = np.mean(pred, axis=0)
        x_pred, y_pred = pred
        print(OR,n,"predicted",x_pred,y_pred, file=f, sep="\t")
        if OR in ORs:
            print(OR, n, "slide-seq", coords[OR]["X"], coords[OR]["Y"], file=f, sep="\t")

Loading coordinates
77 ORs with Slide-seq coordinates
                           PC_1       PC_2      PC_3      PC_4      PC_5  \
V3F1_CAACGGCAGTAAACAC -2.708932  -2.132975 -0.944150 -2.041754  2.190978   
V3F3_GAGACCCTCTGAGCAT -6.120769   5.108752  1.388388 -1.472756  0.176348   
V3F1_GAGGGATGTAGCCCTG -3.594516 -10.143140  3.549887 -0.801045 -2.812715   
V3F3_ACGTAGTCAGGACAGT -5.803491  -0.791259  4.488344  6.078957 -4.919589   
V3M1_ACCAAACCAAGTGACG  1.717312  -3.617910  3.445913  3.360594 -1.074957   

                           PC_6      PC_7      PC_8      PC_9     PC_10  ...  \
V3F1_CAACGGCAGTAAACAC  0.752998  0.678070 -0.105306  1.752985 -0.194909  ...   
V3F3_GAGACCCTCTGAGCAT -0.553542 -0.436527 -0.706083 -0.363342  0.275126  ...   
V3F1_GAGGGATGTAGCCCTG -1.656208  0.475931 -0.764649  1.248286  0.657523  ...   
V3F3_ACGTAGTCAGGACAGT  3.173439  2.533901 -1.802449  1.768489 -1.167335  ...   
V3M1_ACCAAACCAAGTGACG  7.194287 -3.454088 -3.498878  0.760647 -0.652230  ...   

        